Tarea:
Tras mostrar opciones para la detección y extracción de información de caras humanas con deepface, la tarea a entregar consiste en proponer un escenario de aplicación y desarrollar un prototipo de temática libreque provoque reacciones a partir de la información extraida del rostro. Los detectores proporcionan información del rostro, y de sus elementos faciales. Ideas inmediatas pueden ser filtros, aunque no hay limitaciones en este sentido. La entrega debe venir acompañada de un gif animado o vídeo de un máximo de 30 segundos con momentos seleccionados de la propuesta. 

In [2]:
from collections import deque
import cv2
import numpy as np
import mediapipe as mp

# Cargar la imagen del sombrero y el parche
img = cv2.imread('sombrero.jpg')
img2 = cv2.imread('parche.jpg')

# Convertir las imágenes a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Aplicar un umbral para crear imágenes binarias
_, umbral = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY_INV)
_, umbral2 = cv2.threshold(gray2, 240, 255, cv2.THRESH_BINARY_INV)

# Encontrar los contornos en la imagen binaria
contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contornos2, _ = cv2.findContours(umbral2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Crear una máscara vacía
mascara = np.zeros_like(img)
mascara2 = np.zeros_like(img2)

# Rellenar las máscaras con el contorno del objeto (sombrero y parche)
cv2.drawContours(mascara, contornos, -1, (255, 255, 255), thickness=cv2.FILLED)
cv2.drawContours(mascara2, contornos2, -1, (255, 255, 255), thickness=cv2.FILLED)

# Extraer los objetos (sombrero y parche) de las imágenes originales utilizando las máscaras
objeto_extraido = cv2.bitwise_and(img, mascara)
objeto_extraido2 = cv2.bitwise_and(img2, mascara2)

# Inicializar MediaPipe Face Detection para detectar los puntos de la cara
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)

# Inicializar MediaPipe Face Mesh para detectar los puntos de la cara con mayor precision
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.2)

def eye_aspect_ratio(coordinates):
     d_A = np.linalg.norm(np.array(coordinates[1]) - np.array(coordinates[5]))
     d_B = np.linalg.norm(np.array(coordinates[2]) - np.array(coordinates[4]))
     d_C = np.linalg.norm(np.array(coordinates[0]) - np.array(coordinates[3]))
     return (d_A + d_B) / (2 * d_C)

def drawing_output(frame, blink_counter):
     aux_image = np.zeros(frame.shape, np.uint8)
     output = cv2.addWeighted(frame, 1, aux_image, 0.7, 1)
     cv2.rectangle(output, (0, 0), (210, 50), (0, 0, 0), -1)
     cv2.rectangle(output, (212, 0), (265, 50), (0, 0, 0),-1)
     cv2.putText(output, "Num. Parpadeos:", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
     cv2.putText(output, "{}".format(blink_counter), (220, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
     return output


index_left_eye = [33, 160, 158, 133, 153, 144]
index_right_eye = [362, 385, 387, 263, 373, 380]
EAR_THRESH = 0.26
NUM_FRAMES = 2
aux_counter = 0
blink_counter = 0
pts_ear = deque(maxlen=64)
i = 0

# Iniciar la captura de video
vid = cv2.VideoCapture(0)

while(True):

    ret, frame = vid.read()
    if not ret:
        break
    height, width, _ = frame.shape

    #Coodenadas ojos izquierdo y derecho
    coordinates_left_eye = []
    coordinates_right_eye = []

    # Convertir el fotograma a RGB para MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    #Contador de parpadeos
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            for index in index_left_eye:
                x = int(face_landmarks.landmark[index].x * width)
                y = int(face_landmarks.landmark[index].y * height)
                coordinates_left_eye.append([x, y])
            for index in index_right_eye:
                x = int(face_landmarks.landmark[index].x * width)
                y = int(face_landmarks.landmark[index].y * height)
                coordinates_right_eye.append([x, y])
        ear_left_eye = eye_aspect_ratio(coordinates_left_eye)
        ear_right_eye = eye_aspect_ratio(coordinates_right_eye)
        ear = (ear_left_eye + ear_right_eye)/2
        # Ojos cerrados
        if ear < EAR_THRESH:
            aux_counter += 1
        else:
            if aux_counter >= NUM_FRAMES:
                aux_counter = 0
                blink_counter += 1                
        frame = drawing_output(frame, blink_counter)

        if(blink_counter%2!=0):

            results = face_detection.process(frame_rgb)
            if results.detections:
                for detection in results.detections:
                    # Obtener la caja delimitadora (bounding box) de la cara
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                    # Establecer un factor de escala para aumentar el tamaño del sombrero
                    escala_sombrero = 1.3  # Aumentar el tamaño del sombrero en un 30% (ajustar este valor según sea necesario)
                    # Redimensionar la imagen del sombrero para que se ajuste a la cara con un tamaño aumentado
                    overlay = cv2.resize(objeto_extraido, (int(w * escala_sombrero), int(h * escala_sombrero)))
                    # Determinar la nueva posición para colocar el sombrero centrado encima de la cabeza
                    y_offset = int(h * 0)  # Coloca el sombrero por encima de la cabeza
                    x_offset = int(w * 0.5)  # Centrar el sombrero sobre el ancho de la cara
                    # Redefinir las coordenadas para el sombrero centrado sobre la cara
                    new_x = x + (w // 2) - (overlay.shape[1] // 2)  # Centrado horizontalmente
                    new_y = y - overlay.shape[0] + y_offset  # Colocado por encima de la cabeza
                    # Asegurarse de que el sombrero no se mueva fuera de los límites de la imagen
                    new_x = max(new_x, 0)
                    new_y = max(new_y, 0)
                    # Obtener la región del fotograma donde se colocará el sombrero
                    n_frame = frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]]   
                    # Convertir la imagen del sombrero a escala de grises para obtener la máscara
                    gray_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
                    # Aplicar un umbral a la imagen en escala de grises para generar la máscara
                    _, mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)
                    # Invertir la máscara
                    mask_inv = cv2.bitwise_not(mask)  
                    # Extraer la parte de la cara (fondo) que no tiene el sombrero
                    bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)
                    # Extraer la parte del sombrero donde se va a colocar
                    fg_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)
                    # Combinar el fondo de la cara y el sombrero en la misma región
                    result = cv2.add(bg_frame, fg_overlay)
                    # Superponer el resultado en el fotograma original
                    frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]] = result

            results = face_mesh.process(frame_rgb)
            if results.multi_face_landmarks:
                for landmarks in results.multi_face_landmarks:
                    # Obtener las coordenadas del ojo derecho (punto 263 en MediaPipe)
                    ih, iw, _ = frame.shape
                    right_eye = landmarks.landmark[263]  # Ojo derecho (punto 263)
                    # Convertir las coordenadas normalizadas a píxeles
                    right_eye_x, right_eye_y = int(right_eye.x * iw), int(right_eye.y * ih)
                    # Redimensionar el parche para que encaje sobre el ojo derecho
                    overlay = cv2.resize(objeto_extraido2, (80, 80))  # Tamaño del parche
                    h, w, _ = overlay.shape
                    # Determinar la posición para colocar el parche (encima del ojo derecho)
                    y_offset = int(h * 0.5)  # Coloca el parche justo sobre el ojo
                    y = max(right_eye_y - y_offset, 0)  # Asegurarse de que no se mueva fuera de los límites de la imagen
                    x = right_eye_x - w // 2 - 10  # Centrar el parche sobre el ojo derecho
                    # Extraer la región de la cara (en este caso, sobre el ojo derecho)
                    n_frame = frame[y:y+h, x:x+w]
                    # Convertir la imagen del parche a escala de grises para obtener la máscara
                    gray_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
                    # Aplicar un umbral a la imagen en escala de grises para generar la máscara
                    _, mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)
                    # Invertir la máscara
                    mask_inv = cv2.bitwise_not(mask)
                    # Extraer la parte de la cara (fondo) que no tiene el parche
                    bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)
                    # Extraer la parte del parche donde se va a colocar
                    fg_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)
                    # Combinar el fondo de la cara y el parche en la misma región
                    result = cv2.add(bg_frame, fg_overlay)
                    # Superponer el resultado en el fotograma original
                    frame[y:y+h, x:x+w] = result

    # Mostrar el fotograma con el parche superpuesto
    cv2.imshow('Frame', frame)
    
    # Detener la ejecución con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la captura de video y cerrar las ventanas
vid.release()
cv2.destroyAllWindows()


c:\Users\cquevedo\AppData\Local\anaconda3\envs\VC_P1\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
